# User Added to Local Administrators
This rule triggers on user accounts that are added to the local Administrators group, which could be legitimate activity or a sign of privilege escalation activity

## Rule Content
```
- title: User Added to Local Administrators
  id: c265cf08-3f99-46c1-8d59-328247057d57
  description: This rule triggers on user accounts that are added to the local Administrators
    group, which could be legitimate activity or a sign of privilege escalation activity
  status: stable
  author: Florian Roth
  tags:
  - attack.privilege_escalation
  - attack.t1078
  logsource:
    product: windows
    service: security
    category: null
  detection:
    selection:
      EventID: 4732
    selection_group1:
      GroupName: Administrators
    selection_group2:
      GroupSid: S-1-5-32-544
    filter:
      SubjectUserName: '*$'
    condition: selection and (1 of selection_group*) and not filter
  falsepositives:
  - Legitimate administrative activity
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((event_id:"4732" AND (group_name:"Administrators" OR group_sid:"S\-1\-5\-32\-544")) AND (NOT (SubjectUserName.keyword:*$)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()